<a href="https://colab.research.google.com/github/balajigood3/BALAJI-VP/blob/main/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install bitsandbytes accelerate
!pip install transformers peft datasets


In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

# Use REGULAR model - NO quantization problems
model_id = "microsoft/DialoGPT-medium"  # Works instantly

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

# Fix padding
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("✅ Model loaded perfectly!")


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ Model loaded perfectly!


In [9]:
# Tunix-style reasoning test
prompt = """Q: What is 15 + 27?
Think step-by-step: First 15 + 20 = 35, then add 7 more...
A:"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=50,
    temperature=0.7,
    do_sample=True
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: What is 15 + 27?
Think step-by-step: First 15 + 20 = 35, then add 7 more...
A: 1


In [10]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Ready for LoRA training
print("🚀 GPT-2 loaded - no errors!")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

🚀 GPT-2 loaded - no errors!


In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

# Use OPEN model (no auth needed) - Mistral 7B
model_id = "microsoft/DialoGPT-medium"  # OR "gpt2" OR "facebook/opt-1.3b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Fix padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("✅ Model loaded without tokens!")

# LoRA config (works on any model)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["c_attn", "c_proj"],  # DialoGPT targets
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# Sample reasoning dataset (no Kaggle needed)
from datasets import Dataset
data = {
    "question": ["What is 15+27?", "Capital of France?"],
    "reasoning": ["15+20=35, plus 7 more is 42", "France's largest city and government seat"],
    "answer": ["42", "Paris"]
}
dataset = Dataset.from_dict(data)

def format_prompt(example):
    return f"Q: {example['question']}\nThink: {example['reasoning']}\nA: {example['answer']}"

train_dataset = dataset.map(lambda x: {"text": format_prompt(x)})

print("🚀 Ready for Tunix-style training!")
print(train_dataset[0]['text'])


✅ Model loaded without tokens!


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2285: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

🚀 Ready for Tunix-style training!
Q: What is 15+27?
Think: 15+20=35, plus 7 more is 42
A: 42
